In [ ]:
%matplotlib inline

from hydromt.log import setuplog
from hydromt_sfincs import SfincsModel, utils
from hydromt.config import configread
import geopandas as gpd

import subprocess

import hydromt
from hydromt import DataCatalog
import datetime
import xarray as xr
import sys, os
from os.path import join
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, date
# sys.path.append('./')
# import HYDmodfunctions as HM

## Hydraulic simulations
Functions to perform the hydraulic simulations

The loop for the 25 simulations

Calculate and store the response function

Create the flood maps and the water depths


In [ ]:
# STEP 1 - WE CONSTRUCT THE BASE MODEL
# Building the base model - this will create the base sfincs model without the boundary conditions yet
# Location from Maria : The coordinates: Latitude 49°59'37.31"N, Longitude   8°18'6.57"E (UTM zone 32N: 449957.51 m E, 5538163.45 m N)
!hydromt build sfincs "../confluence_base" "{'geom': '../data/boundary.shp'}" -r 200 -i ../data/riverine_setup.ini --dd -d "../data/data_catalog.yml" -vv 


In [ ]:
def step2():
    # STEP 2 - WE CHECK THE BASE MODEL
    # We check the model
    root = r"../confluence_base"
    logger = setuplog('update', os.path.join(root, "hydromt.log"), log_level=10)
    mod = SfincsModel(root=root, mode="r", logger = logger)
    mod.read()

    #To check the model you created through Python. Can also use the gis folder
    mod.plot_basemap(
        geoms=["src", "rivers"],
        figsize=(14, 14 * 0.65))
    return mod

In [ ]:
#Step3
def step3(mod, depth1, depth2, rp1, rp2, fn):
    #STEP 3 - WE ADD THE BOUNDARY CONDITIONS - THE DISCHARGES
    #We read the new time series and save to the model 
    # fn = r'C:\Users\Fonse_ma\OneDrive - Stichting Deltares\Desktop\Maria\Notebooks\RQ3\data\test_discharges.csv'
    # fn = r'C:\Users\Fonse_ma\OneDrive - Stichting Deltares\Desktop\Maria\Notebooks\RQ3\data\TS' + TSpath #\TS_Qmean.csv'
    # fn = r'P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ3\data\TS' + TSpath
    dis = pd.read_csv(fn, index_col='date', parse_dates=True)
    gdf_q = mod.forcing['dis'].vector.to_gdf()
    mod.set_forcing_1d(ts=dis.copy(), xy=gdf_q, name='discharge')

    #We ma  ke the dep file deeper at the rivers
    geotiff_path = r'P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ3\confluence_base\gis\COPY.tif'
    # Open into an xarray.DataArray
    geotiff_da = xr.open_rasterio(geotiff_path)
    mod.set_staticmaps(geotiff_da, name='rivmsk2')

    current_dep = mod.staticmaps['dep'].copy()
    msk = mod.staticmaps['rivmsk2']==1  # mod.staticmaps['rivmsk']==1
    msk2 = mod.staticmaps['rivmsk2']==2  # mod.staticmaps['rivmsk']==1
    new_dep = xr.where(msk[0,:,:], current_dep - depth1, current_dep)
    new_dep = xr.where(msk2[0,:,:], new_dep - depth2, new_dep)
    new_dep.attrs = current_dep.attrs
    mod._staticmaps['dep'] = new_dep
    mod.set_root(r"..\confluence_base_dep")
    mod.write_staticmaps()

    tstart = dis.index[0].strftime("%Y%m%d %H%M%S") 
    tstop = dis.index[-1].strftime("%Y%m%d %H%M%S") 

    mod.setup_config(**{
    'tref': tstart, 
    'tstart': tstart, 
    'tstop': tstop})

    # We set initial water depth in the river channel (called zsini or a restart file in SFINCS)
    mask = np.logical_or(mod.staticmaps['rivmsk']==1, mod.staticmaps['dep']<0)
    zsini = mod.staticmaps['dep'].where(~mask, np.maximum(mod.staticmaps['dep']+0.2, 0.5)).where(mod.mask!=0,0)
    zsini.raster.set_nodata(0)
    mod.set_states(zsini, 'zsini')
    mod.config.pop('zsini',None)

    #We update the model and save it in a new folder. Note that we only save the changed elements, for the rest we point back 
    # to the base model
    # rp1, rp2 = 3, 3
    mod.set_root(r"..\confluence_run_S{}_S{}".format(rp1,rp2))
    # rp1 = 0.8   #input of the dunction
    # mod.set_root(r"..\confluence_run_Q{}".format(rp1))
    mod.write_forcing()
    mod.write_config(rel_path=f'../confluence_base_dep') #confluence_base
    mod.write_states()  
    return

In [ ]:
#Step4
#%% STEP 4 -  We run the SFINCS Model 
def step4(rp1, rp2):
    #STEP 4 -  We run the SFINCS Model 
    #Note: it took 940 sec (about 15 min) on my crappy computer to run 2 weeks of daily flows. For 4 days this was around 3 minutes
    # rp1, rp2 = 3, 3
    folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
    # rp1 = 0.8
    # folder = r"..\confluence_run_Q{}".format(rp1)

    # fn_exe = "C:/CODE/CoastalRiver/subgrid_openacc_11_rev295_16092021/sfincs.exe"
    # fn_exe="C:/Users/Fonse_ma/OneDrive - Stichting Deltares/Desktop/Maria/Notebooks/RQ3/data/subgrid_openacc_11_rev295_16092021/sfincs.exe"
    fn_exe="P:/11206883-006-dar-cloud-computing/GRADE_export/HYDa/RQ3/data/subgrid_openacc_11_rev295_16092021/sfincs.exe"

    with open(os.path.join(folder, "sfincs.log"), 'w') as f:
        p = subprocess.Popen([fn_exe], stdout=f, cwd=folder)
        p.wait()

    mod = SfincsModel(folder, mode='r')
    mod.read_results()

    da_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

    #Results will be stored in 
    mod.write_raster("results.hmax", compress="LZW")
    mod.write_raster("results.zsmax", compress="LZW")
    return

In [ ]:
#Figure: Inundation map
def Fig(rp1, rp2):
    tittle = f'S{rp1}_S{rp2}'

    folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
    mod = SfincsModel(folder, mode='r')

    fig, ax = mod.plot_basemap(
        fn_out=None,
        variable=None,
        bmap="sat",
        geoms=["src", "obs"],
        plot_bounds=False,
        figsize=(11, 7),
    )

    hmin = 0.2  # minimum flood depth [m] to plot

    da_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

    da_hmax_fld = da_hmax.where(da_hmax > hmin)
    # da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
    da_hmax.attrs.update(long_name="flood depth", unit="m")

    # plot overland flooding based on gswo mask and mimum flood depth
    cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
    cax_fld = da_hmax_fld.plot(
        ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs
    )

    ax.set_title(f"SFINCS maximum water depth | run {tittle}")
    # cax_fld.savefig(folder + r'hmax.png')
    return

In [ ]:
#Figures: Depth
def FigDepth(rp1, rp2):
    tittle = f'S{rp1}_S{rp2}'

    folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
    mod = SfincsModel(folder, mode='r')

    da_h = mod.results["h"]
    ts_S2c = da_h.sel(x=451714.481,y=5539238.728, method='nearest').to_dataframe() #.drop(['x','y','spatial_ref'])
    ts_S2u = da_h.sel(x=485304.5621,y=5553874.868, method='nearest').to_dataframe() #.drop(['x','y','spatial_ref'])

    ts_S1c = da_h.sel(x=451313.526,y=5536064.001, method='nearest').to_dataframe() #.drop(['x','y','spatial_ref'])
    ts_S1u = da_h.sel(x=458653.473,y=5486412.710, method='nearest').to_dataframe() #.drop(['x','y','spatial_ref'])

    ts_Cc = da_h.sel(x=448395.390,y=5539294.143, method='nearest').to_dataframe() #.drop(['x','y','spatial_ref'])
    ts_Cd = da_h.sel(x=418192.635,y=5539732.335, method='nearest').to_dataframe() #.drop(['x','y','spatial_ref'])

    # ts_inun = da_h.sel(x=453653.688,y=5529329.344, method='nearest').to_dataframe() #.drop(['x','y','spatial_ref'])

    plt.figure(figsize=(7, 5))
    plt.plot(ts_S1c['h'], 'b', label='S1 confluence')
    plt.plot(ts_S1u['h'], 'g', label='S1 upstream')
    plt.xticks(rotation = 90)
    plt.title(f'SFINCS water depth S1 | run {tittle}')
    plt.legend(fontsize=18, bbox_to_anchor=(1,1), loc="upper left")
    plt.savefig(folder + r'\wd_S1.png')

    plt.figure(figsize=(7, 5))
    plt.plot(ts_S2c['h'], 'b', label='S2 confluence')
    plt.plot(ts_S2u['h'], 'g', label='S2 upstream')
    plt.xticks(rotation = 90)
    plt.title(f'SFINCS water depth S2 | run {tittle}')
    plt.legend(fontsize=18, bbox_to_anchor=(1,1), loc="upper left")
    plt.savefig(folder + r'\wd_S2.png')

    plt.figure(figsize=(7, 5))
    plt.plot(ts_Cc['h'], 'b', label='C confluence')
    plt.plot(ts_Cd['h'], 'g', label='C downstream')
    plt.xticks(rotation = 90)
    plt.title(f'SFINCS water depth C | run {tittle}')
    plt.legend(fontsize=18, bbox_to_anchor=(1,1), loc="upper left")
    plt.savefig(folder + r'\wd_C.png')

    # plt.figure()
    # plt.plot(ts_inun['h'], label='Close to flooded area')
    # plt.xticks(rotation = 90)
    # plt.title(f'SFINCS water depth | Inundated area')
    # plt.legend()
    return

In [ ]:
def response(rp1,rp2):
    folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
    mod = SfincsModel(folder, mode='r')

    geotiff_path = r'P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ3\confluence_base\gis\rivmsk.tif'
    # Open into an xarray.DataArray
    geotiff_da = xr.open_rasterio(geotiff_path)
    mod.set_staticmaps(geotiff_da, name='rivmsk')

    msk = mod.staticmaps['rivmsk']==1
    da_hmax = mod.results["hmax"]
    hmin = 0.2 
    da_hmax_fld = da_hmax.where(~msk).where(da_hmax > hmin)
    da_hmax_fld

    da_hmax.attrs.update(long_name="flood depth", unit="m")
    numgrid = np.count_nonzero(~np.isnan(da_hmax_fld.values)) #* 0.200 * 0.200 #km2
    area = numgrid * 200 * 200 #m2

    vol = da_hmax_fld.values * 200 * 200 # m3
    Vol_total = np.nansum(vol)#/(1000*1000*1000)   #m3
    # Vol_total2 = sum(vol[~np.isnan(vol)])#/(1000*1000*1000)   #m3

    depth_mean = np.nanmean(da_hmax_fld.values)
    return area, Vol_total, depth_mean

In [ ]:
# Response grids
#Discharges S1, S2
#names
S1_n = ['S0', 'S1', 'S2', 'S3', 'S4']   #Colum
S2_n = ['S0', 'S1', 'S2', 'S3', 'S4']   #INDEX
Res_Area = pd.DataFrame(index=[S2_n] ,columns=[S1_n])
Res_Volume = pd.DataFrame(index=[S2_n] ,columns=[S1_n])
Res_mDepth = pd.DataFrame(index=[S2_n] ,columns=[S1_n])

In [ ]:

for s1 in range(len(S1_n)): #(len(S1_n))   (0, 1)
    for s2 in range(len(S2_n)): #(len(S2_n))
        depth1, depth2 = 26.7, 14.1 # (26.5,27)
        rp1 = s1 #from S1 [0, 1, 2, 3, 4]
        rp2 = s2 #from S2 [0, 1, 2, 3, 4]
        TSpath = r'\TS_S%d' %rp1 + r'_S%d.csv' %rp2 #  according to Q1_Q2
        fn = r'P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ3\data\TSgrid' + TSpath
        print(f'Starting {TSpath} at {str(datetime.now())}')

        mod = step2()
        print(f'Done Step2 at {str(datetime.now())}')
        step3(mod, depth1, depth2, rp1, rp2, fn)
        print(f'Done Step3 at {str(datetime.now())}')
        step4(rp1, rp2)
        print(f'Done Step4 at {str(datetime.now())}')
        # Fig(rp1, rp2)
        # FigDepth(rp1, rp2)
        area, Vol_total, depth_mean = response(rp1, rp2)

        Res_Area.iloc[s2,s1] = area
        Res_Volume.iloc[s2,s1] = Vol_total
        Res_mDepth.iloc[s2,s1] = depth_mean
        print(f'Finishing {TSpath} at {str(datetime.now())}')

In [ ]:
S1 = np.array([  723.95,  3403.62,  6083.29,  8762.96, 11442.62])
S2 = np.array([  27.14, 1119.63, 2212.12, 3304.61, 4397.12])

In [ ]:
S1_n = ['S0', 'S1', 'S2', 'S3', 'S4']   #Colum
S2_n = ['S0', 'S1', 'S2', 'S3', 'S4']   #INDEX

In [ ]:
plt.rcParams['font.size'] = '16'

import matplotlib.colors
YlGrBl = matplotlib.colors.LinearSegmentedColormap.from_list("", ["blue", "yellow", "red"])


In [ ]:
x, y = np.meshgrid(S1, S2)
z = Res_Area.values / 1000000
cmap = 'YlOrRd'
unit = 'm3/s'

plt.figure(figsize=(6, 5)) 
plt.contourf(x, y, z, cmap=cmap)
# plt.colorbar()
cbar = plt.colorbar(label="Area of inundation (km2)", orientation="vertical") #, fontsize = 16)
cbar.ax.tick_params(labelsize=12)
# cbar.ax.set_label(size='large')
plt.xticks(fontsize = 12, rotation = 90)
plt.yticks(fontsize = 12)
plt.title(f'Response Function\nArea of inundation', fontsize = 20) 
plt.xlabel(f'S1 ({unit})', fontsize=16)
plt.ylabel(f'S2 ({unit})', fontsize=16)
# plt.grid()    
# plt.show()

In [ ]:
x, y = np.meshgrid(S1, S2)
z = Res_Volume.values / 1000000000

cmap = 'YlOrRd'
unit = 'm3/s'

plt.figure(figsize=(6, 5)) 
plt.contourf(x, y, z, cmap=cmap)
# plt.colorbar()
cbar = plt.colorbar(label="Volume of inundation (km3)", orientation="vertical") #, fontsize = 16)
cbar.ax.tick_params(labelsize=12)
# cbar.ax.set_label(size='large')
plt.xticks(fontsize = 12, rotation = 90)
plt.yticks(fontsize = 12)
plt.title(f'Response Function\nVolume of inundation', fontsize = 20) 
plt.xlabel(f'S1 ({unit})', fontsize=16)
plt.ylabel(f'S2 ({unit})', fontsize=16)
# plt.grid()    
# plt.show()

In [ ]:
TSpath = r'\TS_S%d' %rp1 + r'_S%d.csv' %rp2 #  according to Q1_Q2
fn = r'P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ3\Response_functions' 

sv = r'\Res_Area.pkl'
Res_Area.to_pickle(fn + sv)
sv = r'\Res_Volume.pkl'
Res_Volume.to_pickle(fn + sv)
sv = r'\Res_mDepth.pkl'
Res_mDepth.to_pickle(fn + sv)


In [ ]:
# Res_Area

# conf.to_csv('Confluences_wnames2.csv', sep=';')
# conf = pd.read_csv('Confluences_wnames.csv', delimiter=';')

fn = r'P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ3\Response_functions' 

sv = r'\Res_Area.csv'
Res_Area.to_csv(fn + sv, sep=';')
sv = r'\Res_Volume.csv'
Res_Volume.to_csv(fn + sv, sep=';')
sv = r'\Res_mDepth.csv'
Res_mDepth.to_csv(fn + sv, sep=';')


In [ ]:
#Figure: Inundation map
def Fig(rp1, rp2):
    tittle = f'S{rp1}_S{rp2}'

    folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
    mod = SfincsModel(folder, mode='r')

    fig, ax = mod.plot_basemap(
        fn_out=None,
        variable=None,
        bmap="sat",
        geoms=["src", "obs"],
        plot_bounds=False,
        figsize=(11, 7),
    )

    hmin = 0.2  # minimum flood depth [m] to plot

    da_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

    da_hmax_fld = da_hmax.where(da_hmax > hmin)
    # da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
    da_hmax.attrs.update(long_name="flood depth", unit="m")

    # plot overland flooding based on gswo mask and mimum flood depth
    cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
    cax_fld = da_hmax_fld.plot(
        ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs
    )

    ax.set_title(f"SFINCS maximum water depth | run {tittle}", y=1.05, fontsize=20)
    # cax_fld.savefig(folder + r'hmax.png')
    plt.savefig(join(mod.root, 'figs', 'hmax.png'), dpi=225, bbox_inches="tight")
    return

In [ ]:
#Figures: Depth
def FigDepth(rp1, rp2):
    tittle = f'S{rp1}_S{rp2}'

    folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
    mod = SfincsModel(folder, mode='r')

    da_h = mod.results["h"]
    ts_S2c = da_h.sel(x=451714.481,y=5539238.728, method='nearest').to_dataframe() #.drop(['x','y','spatial_ref'])
    ts_S2u = da_h.sel(x=485304.5621,y=5553874.868, method='nearest').to_dataframe() #.drop(['x','y','spatial_ref'])

    ts_S1c = da_h.sel(x=451313.526,y=5536064.001, method='nearest').to_dataframe() #.drop(['x','y','spatial_ref'])
    ts_S1u = da_h.sel(x=458653.473,y=5486412.710, method='nearest').to_dataframe() #.drop(['x','y','spatial_ref'])

    ts_Cc = da_h.sel(x=448395.390,y=5539294.143, method='nearest').to_dataframe() #.drop(['x','y','spatial_ref'])
    ts_Cd = da_h.sel(x=418192.635,y=5539732.335, method='nearest').to_dataframe() #.drop(['x','y','spatial_ref'])

    # ts_inun = da_h.sel(x=453653.688,y=5529329.344, method='nearest').to_dataframe() #.drop(['x','y','spatial_ref'])

    plt.figure(figsize=(10, 5))
    plt.plot(ts_S1c['h'], 'b', label='S1 confluence')
    plt.plot(ts_S1u['h'], 'g', label='S1 upstream')
    plt.xticks(rotation = 45)
    plt.title(f'SFINCS water depth S1 | run {tittle}', fontsize=20)
    plt.legend(fontsize=16, bbox_to_anchor=(1,1), loc="upper left")
    plt.tight_layout()
    plt.savefig(folder + r'\figs\wd_S1.png', bbox_inches="tight")

    plt.figure(figsize=(10, 5))
    plt.plot(ts_S2c['h'], 'b', label='S2 confluence')
    plt.plot(ts_S2u['h'], 'g', label='S2 upstream')
    plt.xticks(rotation = 45)
    plt.title(f'SFINCS water depth S2 | run {tittle}', fontsize=20)
    plt.legend(fontsize=16, bbox_to_anchor=(1,1), loc="upper left")
    plt.tight_layout()
    plt.savefig(folder + r'\figs\wd_S2.png', bbox_inches="tight")

    plt.figure(figsize=(10, 5))
    plt.plot(ts_Cc['h'], 'b', label='C confluence')
    plt.plot(ts_Cd['h'], 'g', label='C downstream')
    plt.xticks(rotation = 45)
    plt.title(f'SFINCS water depth C | run {tittle}', fontsize=20)
    plt.legend(fontsize=16, bbox_to_anchor=(1,1), loc="upper left")
    plt.tight_layout()
    plt.savefig(folder + r'\figs\wd_C.png', bbox_inches="tight")

    # plt.figure()
    # plt.plot(ts_inun['h'], label='Close to flooded area')
    # plt.xticks(rotation = 90)
    # plt.title(f'SFINCS water depth | Inundated area')
    # plt.legend()
    return

In [ ]:
def FigForcing(rp1, rp2):
    tittle = f'S{rp1}_S{rp2}'

    folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
    mod = SfincsModel(folder, mode='r')
    mod.plot_forcing()
    return

In [ ]:

plt.rcParams['font.size'] = '13'
plt.rcParams['xtick.labelsize'] = '10'
plt.rcParams['savefig.facecolor']='white'

for s1 in range(len(S1_n)): #(len(S1_n))   (0, 1)
    for s2 in range(len(S2_n)): #(len(S2_n))
        rp1 = s1 #from S1 [0, 1, 2, 3, 4]
        rp2 = s2 #from S2 [0, 1, 2, 3, 4]
        TSpath = r'\TS_S%d' %rp1 + r'_S%d.csv' %rp2 #  according to Q1_Q2
        fn = r'P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ3\data\TSgrid' + TSpath
        print(f'Starting {TSpath} at {str(datetime.now())}')
        FigForcing(rp1, rp2)
        Fig(rp1, rp2)
        FigDepth(rp1, rp2)
        print(f'Finishing {TSpath} at {str(datetime.now())}')